# REDDY

Load modules and initialize objects

In [3]:
from modules.reddit_api import RedditAPI
from modules.cache_handler import Cache
from credentials import client_id, secret
from progressbar import ProgressBar, SimpleProgress, ETA

api   = RedditAPI()
cache = Cache(verbose=False)

api.authenticate(client_id, secret)

# name of the subreddit to analyse
subreddit_name = "datascience"

Download the posts and index them

In [4]:
after = None
count = 0
pbar = ProgressBar(widgets=[SimpleProgress(), ', ', ETA(), "\n"])
iterations = pbar(range(10))

for i in iterations:
    posts, before, after = api.new_posts(subreddit_name, limit=100, after=after, count=count)
    count += len(posts)
    cache.add(posts, overwrite=False)

cache.update_all_indices()

NameError: name 'ProgressBar' is not defined